In [14]:
import os
from dotenv import load_dotenv

from langchain_groq import ChatGroq
from langchain_community.document_loaders import PyPDFLoader
from langchain.chains.summarize import load_summarize_chain
from langchain_core.prompts import PromptTemplate

# Stuff Chain Document

In [15]:
load_dotenv(dotenv_path= r"D:\PYTHON\GENERATIVE AI\.ENV FILES\.env")
Groq_API_Key = os.getenv("GROQ_API_KEY")

In [16]:
Loader = PyPDFLoader("apjspeech.pdf")
Documents = Loader.load_and_split()
Documents

[Document(metadata={'producer': 'GPL Ghostscript 8.15', 'creator': 'PScript5.dll Version 5.2', 'creationdate': 'D:20070730160943', 'moddate': 'D:20070730160943', 'title': 'Microsoft Word - Document1', 'author': 'Shri', 'source': 'apjspeech.pdf', 'total_pages': 7, 'page': 0, 'page_label': '1'}, page_content='A P J Abdul Kalam Departing speech \n \n \nFriends, I am delighted to address you all, in the country and those livi ng abroad, after \nworking with you and completing five beautiful and eventful years in Rashtrapati \nBhavan. Today, it is indeed a thanks giving occasion. I would like to narr ate, how I \nenjoyed every minute of my tenure enriched by the wonderful assoc iation from each one \nof you, hailing from different walks of life, be it politics, sci ence and technology, \nacademics, arts, literature, business, judiciary, administration, local bodies, farming, \nhome makers, special children, media and above all from the youth and st udent \ncommunity who are the future wealt

In [17]:
LLM_Model = ChatGroq(api_key= Groq_API_Key, model="llama3-70b-8192")

In [18]:
Template = """Write a brief and concise summary of the following speech,
speech:{Text}
"""

Prompt = PromptTemplate(input_variables=["Text"], template=Template)

In [19]:
Chain = load_summarize_chain(
    llm=LLM_Model,
    prompt=Prompt,
    chain_type="stuff",
    document_variable_name="Text",
    verbose=True
)
Summary =  Chain.run(Documents)
Summary



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Write a brief and concise summary of the following speech,
speech:A P J Abdul Kalam Departing speech 
 
 
Friends, I am delighted to address you all, in the country and those livi ng abroad, after 
working with you and completing five beautiful and eventful years in Rashtrapati 
Bhavan. Today, it is indeed a thanks giving occasion. I would like to narr ate, how I 
enjoyed every minute of my tenure enriched by the wonderful assoc iation from each one 
of you, hailing from different walks of life, be it politics, sci ence and technology, 
academics, arts, literature, business, judiciary, administration, local bodies, farming, 
home makers, special children, media and above all from the youth and st udent 
community who are the future wealth of our country. During my intera ction at 
Rashtrapati Bhavan in Delhi and at every state and union territor y as well as through my 
online inte

"Here is a brief and concise summary of the departing speech of A.P.J. Abdul Kalam:\n\nIn his farewell address, Dr. A.P.J. Abdul Kalam shared his experiences and interactions with people from various walks of life during his five-year tenure as President of India. He highlighted 10 important messages, including the need to accelerate development, empower villages, and promote rural-urban connectivity. He also emphasized the importance of youth empowerment, education, and value-based leadership. Dr. Kalam expressed his confidence in India's ability to become a developed nation by 2020, citing the country's natural resources, hardworking people, and the power of its 540 million youth. He concluded by emphasizing the need for collective effort to achieve this goal and make India a proud and prosperous nation."

# Map Reduce

In [20]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [21]:
Final_split_Documents = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=100).split_documents(Documents)
Final_split_Documents

[Document(metadata={'producer': 'GPL Ghostscript 8.15', 'creator': 'PScript5.dll Version 5.2', 'creationdate': 'D:20070730160943', 'moddate': 'D:20070730160943', 'title': 'Microsoft Word - Document1', 'author': 'Shri', 'source': 'apjspeech.pdf', 'total_pages': 7, 'page': 0, 'page_label': '1'}, page_content='A P J Abdul Kalam Departing speech \n \n \nFriends, I am delighted to address you all, in the country and those livi ng abroad, after \nworking with you and completing five beautiful and eventful years in Rashtrapati \nBhavan. Today, it is indeed a thanks giving occasion. I would like to narr ate, how I \nenjoyed every minute of my tenure enriched by the wonderful assoc iation from each one \nof you, hailing from different walks of life, be it politics, sci ence and technology, \nacademics, arts, literature, business, judiciary, administration, local bodies, farming, \nhome makers, special children, media and above all from the youth and st udent \ncommunity who are the future wealt

In [23]:
# Define the map prompt
Template = """Write a brief and concise summary of the following speech,
speech:{Text}
"""
Prompt = PromptTemplate(input_variables=["Text"], template=Template)

# Define the combine prompt
Final_Template = """Give us a brief and concise summary of the following speech with a motivational Title,
speech: {Text}
"""
Final_Prompt = PromptTemplate(input_variables=["Text"], template=Final_Template)

In [24]:
Chain = load_summarize_chain(
    llm=LLM_Model,
    chain_type="map_reduce",
    map_prompt=Prompt,
    combine_prompt=Final_Prompt,
    map_reduce_document_variable_name="Text",
    combine_document_variable_name="Text",
    verbose=True
)

# Run the chain
Summary = Chain.run(Final_split_Documents)
print(Summary)



> Entering new MapReduceDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Write a brief and concise summary of the following speech,
speech:A P J Abdul Kalam Departing speech 
 
 
Friends, I am delighted to address you all, in the country and those livi ng abroad, after 
working with you and completing five beautiful and eventful years in Rashtrapati 
Bhavan. Today, it is indeed a thanks giving occasion. I would like to narr ate, how I 
enjoyed every minute of my tenure enriched by the wonderful assoc iation from each one 
of you, hailing from different walks of life, be it politics, sci ence and technology, 
academics, arts, literature, business, judiciary, administration, local bodies, farming, 
home makers, special children, media and above all from the youth and st udent 
community who are the future wealth of our country. During my intera ction at 
Rashtrapati Bhavan in Delhi and at every state and union territor y as well as through my 
online 

# Refine Chain Summary

In [27]:
initial_template = """Write a brief and concise summary of the following speech:
speech: {text}
"""
initial_prompt = PromptTemplate(input_variables=["text"], template=initial_template)

In [28]:
refine_template = """Your task is to refine the existing summary by incorporating additional details from the new text provided.
Existing summary: {existing_answer}
New text: {text}
Refined summary:"""
refine_prompt = PromptTemplate(input_variables=["existing_answer", "text"], template=refine_template)

In [29]:
Chain = load_summarize_chain(
    llm=LLM_Model,
    chain_type="refine",
    question_prompt=initial_prompt,
    refine_prompt=refine_prompt,
    document_variable_name="text",
    initial_response_name="existing_answer",
    verbose=True
)

# Run the chain
Summary = Chain.run(Final_split_Documents)
print(Summary)



> Entering new RefineDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Write a brief and concise summary of the following speech:
speech: A P J Abdul Kalam Departing speech 
 
 
Friends, I am delighted to address you all, in the country and those livi ng abroad, after 
working with you and completing five beautiful and eventful years in Rashtrapati 
Bhavan. Today, it is indeed a thanks giving occasion. I would like to narr ate, how I 
enjoyed every minute of my tenure enriched by the wonderful assoc iation from each one 
of you, hailing from different walks of life, be it politics, sci ence and technology, 
academics, arts, literature, business, judiciary, administration, local bodies, farming, 
home makers, special children, media and above all from the youth and st udent 
community who are the future wealth of our country. During my intera ction at 
Rashtrapati Bhavan in Delhi and at every state and union territor y as well as through my 
online in